<a href="https://colab.research.google.com/github/fzantalis/colab_collection/blob/master/song_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.5.0+cu121 torchvision==0.20.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121
!pip install gradio transformers diffusers accelerate bitsandbytes
!pip install sentencepiece

In [ ]:
import torch
print(torch.__version__)


In [ ]:
!huggingface-cli login --token <replace_with_your_huggingface_token>

In [ ]:
!rm -rf TotoroUI
!git clone -b totoro3 https://github.com/camenduru/ComfyUI TotoroUI
%cd /workspace/mywork/fzantalis/stable_diffusion/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d models/clip -o t5xxl_fp8_e4m3fn.safetensors

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_post_processing
from totoro import model_management

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
VAEEncode = NODE_CLASS_MAPPINGS["VAEEncode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
ImageScaleToTotalPixels = nodes_post_processing.NODE_CLASS_MAPPINGS["ImageScaleToTotalPixels"]()

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

In [ ]:
!wget https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -O /usr/local/bin/yt-dlp
!chmod a+rx /usr/local/bin/yt-dlp

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSpeechSeq2Seq, AutoProcessor
import subprocess
import os
import gc  # Import garbage collector module
import torch
from PIL import Image

#USE THIS IF YOU WANT TO AUTO DOWNLOAD THE SONG FROM YOUTUBE AND EXTRACT LYRICS TO ENGLISH AUTOMATICALLY WITH WHISPER
youtube_link="https://www.youtube.com/watch?v=D0Dx8LvX9Fs"
phrase_num=5
def extract_phrases(response, type):
    phrases = []
    lines = response.strip().split('\n')
    lines = lines[14:]
    stylized_section = False
    for line in lines:
        line = line.strip()
        if not line:
            continue
        # Check if we've reached the "STYLIZED PHRASES:" section
        if line.upper() == type + ":":
            stylized_section = True
            continue
        # Skip lines until we reach the "STYLIZED PHRASES:" section
        if not stylized_section:
            continue
        # Extract phrases in the "STYLIZED PHRASES:" section
        if line[0].isdigit() and line[1] == '.':
            # Remove numbering and any leading/trailing whitespace
            phrase = line.split('.', 1)[1].strip()
            phrases.append(phrase)
            # Stop after collecting 4 phrases
            if len(phrases) >= phrase_num:
                break
    return phrases

import gc
import subprocess

# Define the output file
output_file = "downloaded_audio.mp3"
!rm  $output_file
# Download the audio using yt-dlp
# Build the command as a list of arguments
yt_dlp_command = [
    "yt-dlp",
    "--restrict-filenames",
    "-x",
    "--audio-format",
    "mp3",
    "--output",
    output_file,
    youtube_link
]
#yt-dlp --restrict-filenames -x --audio-format mp3 --output $output_file $youtube_link

# Run the command
result = subprocess.run(yt_dlp_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

if result.returncode != 0:
    # There was an error downloading the audio
    error_message = result.stderr.decode('utf-8')
    print(f"Error downloading audio: {error_message}")
# Now, pass the audio file to Whisper
audio_input = output_file

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v2"

# Load the model
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

# Create the pipeline
whisper_model = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# Transcribe the audio
transcription = whisper_model(audio_input, generate_kwargs={"language": "english", "return_timestamps": True})["text"]
lyrics = transcription
print(lyrics)

# Clean up
del whisper_model.model
del whisper_model.tokenizer
del whisper_model
torch.cuda.empty_cache()
gc.collect()


In [ ]:
# REPLACE THE TEXT BELOW WITH ENGLISH LYRICS IF YOU WANT TO SKIP THE ABOVE STEP
lyrics = """PAST HERE ENGLISH LYRICS
IN PLAIN TEXT IF YOU WANT TO SKIP THE DOWNLOAD
PART ABOVE WITH THE WHISPER MODEL.
THIS IS FASTER BUT YOU NEED TO HAVE THE LYRICS IN ENGLISH"""

In [ ]:
# Load LLaMA model inside the function using the new pipeline method
model_id = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
text_gen_pipeline = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)
phrase_num=5
# Use LLaMA model to extract 4 best phrases
prompt = f"""This is a conversation between a chatbot expert in arts and a user. The user will provide lyrics to the chatbot and the chatbot will extract the {phrase_num} best most inspiring phrases from the lyrics.
Don't hesitate to extract longer phrases in order to maintain
Then the chatbot will try to interpret the meaning of the phrases in order to transform them into a prompt for a text to image generation model.
The chatbot will only provide a list of {phrase_num} phrases, stylized as asked by the user, focusing on the artistic aspects of the picture. The output will be like that:
ORIGINAL PHRASES:
1. example phrase 1
2. example phrase 2
...

STYLIZED PHRASES:
1. stylized phrase 1
2. stylized phrase 2
...

User: Hi chatbot I need some help.
Chatbot: Hi user, how can I help you?

User: Extract the {phrase_num} best most inspiring phrases from the following song lyrics:

{lyrics}

Then rephrase them to be suitable as prompts for a text-to-image model.
For example, if the phrase is 'Broken like a membrane, we burst and we burst one by one'
you could rephrase it like 'A watercolor painting of a broken membrane symbolizing fragility, with figures bursting into stardust one by one, ethereal atmosphere, soft glow lighting, in the style of Salvador Dalí, high definition, intricate details, trending on ArtStation.'
Provide the {phrase_num} phrases as a numbered list.
Don't be too verbose. Only give me back the list of the {phrase_num} phrases.
ra
Chatbot:
"""

response = text_gen_pipeline(
    prompt,
    max_new_tokens=1000,
    do_sample=True,
    temperature=0.7,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    num_return_sequences=1,
)[0]['generated_text']

# Free up memory used by the text generation pipeline
del text_gen_pipeline.model
del text_gen_pipeline.tokenizer
del text_gen_pipeline
del tokenizer
torch.cuda.empty_cache()
gc.collect()

orig_phrases = extract_phrases(response, "ORIGINAL PHRASES")

# Debug: Print the phrases to console
print("Extracted Phrases:")
for idx, orig_phrase in enumerate(orig_phrases, 1):
    print(f"{idx}. {orig_phrase}")

phrases = extract_phrases(response, "STYLIZED PHRASES")
# Debug: Print the phrases to console
print("Stylized Phrases:")
for idx, phrases in enumerate(phrases, 1):
    print(f"{idx}. {phrases}")



In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSpeechSeq2Seq, AutoProcessor
import subprocess
import os
import gc  # Import garbage collector module
import torch
from PIL import Image

counter = 0
phrases = extract_phrases(response, "STYLIZED PHRASES")
for positive_prompt in phrases:
    with torch.inference_mode():
        width = 1024
        height = 1024
        seed = 0
        steps = 20
        sampler_name = "euler"
        scheduler = "simple"

        if seed == 0:
            seed = random.randint(0, 18446744073709551615)
        print(seed)

        print(positive_prompt)
        cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
        cond = [[cond, {"pooled_output": pooled}]]
        noise = RandomNoise.get_noise(seed)[0]
        guider = BasicGuider.get_guider(unet, cond)[0]
        sampler = KSamplerSelect.get_sampler(sampler_name)[0]
        sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]
        latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
        sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
        model_management.soft_empty_cache()
        decoded = VAEDecode.decode(vae, sample)[0].detach()
        Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(str(counter)+".png")
        img_array = np.array(decoded * 255, dtype=np.uint8)[0]
        img = Image.fromarray(img_array)
        display(img)
        counter += 1
# Free up memory
torch.cuda.empty_cache()
gc.collect()
